In [21]:
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, cohen_kappa_score, balanced_accuracy_score

In [22]:
# read the pre-processed data all agreed
data = pd.read_csv('EX1_Input_AllAgreed.csv', index_col=0)

# generate the local dataframe for different domains
local_fruit = data.loc[data['Domain']=='fruit']
local_tool = data.loc[data['Domain']=='tool']
local_music = data.loc[data['Domain']=='music']
local_furniture = data.loc[data['Domain']=='furn']
local_garments = data.loc[data['Domain']=='garm']

In [23]:
# define features and target
features = ['ngrams_last_mean',
            'nrdirhypers',
            'nrhypos',
            'nrpartrels_normalised',
            'depthfromtopsynset_normalised',
            'glosslength_normalised',
            'minwordlength',
            'nroflemmas',
            'polyscore_max']
target = ['vote'] # nb / b

In [24]:
# split training set and testing set
split_ratio = 0.8
random_seed = 7 # R
local = local_garments

X_train, X_test, y_train, y_test = train_test_split(local[features], local[target], train_size=split_ratio, random_state=random_seed)

In [25]:
# SMOTE algorithm
smote = SMOTE(random_state=random_seed, k_neighbors=2)
X_train, y_train = smote.fit_resample(X_train, y_train)
print ('Shape of oversampled data: {}'.format(X_train.shape))
print ('Shape of Y: {}'.format(y_train.shape))

Shape of oversampled data: (340, 9)
Shape of Y: (340, 1)


In [26]:
# define random forest model
rf = RandomForestClassifier(random_state=random_seed, max_features="sqrt", n_estimators=500, oob_score=True, criterion='gini', bootstrap=True).fit(X_train, y_train)

<ipython-input-26-0db4f61b13cf>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = RandomForestClassifier(random_state=random_seed, max_features="sqrt", n_estimators=500, oob_score=True, criterion='gini', bootstrap=True).fit(X_train, y_train)


In [27]:
# predict and make score
pipeline = make_pipeline(smote, rf)

y_pred = pipeline.predict(X_test)
results = classification_report(y_test, y_pred, output_dict=True)
results = pd.DataFrame(results).transpose()

kappa = cohen_kappa_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
results['cohen kappa'] = kappa
results['balanced acc'] = balanced_accuracy
results['local'] = list(local['Domain'])[0]

In [28]:
results

,precision,recall,f1-score,support,cohen kappa,balanced acc,local
b,0.666667,0.500000,0.571429,4.000000,0.539185,0.738889,garm
nb,0.956522,0.977778,0.967033,45.000000,0.539185,0.738889,garm
accuracy,0.938776,0.938776,0.938776,0.938776,0.539185,0.738889,garm
macro avg,0.811594,0.738889,0.769231,49.000000,0.539185,0.738889,garm
weighted avg,0.932860,0.938776,0.934739,49.000000,0.539185,0.738889,garm


In [29]:
# importance of features
importance = rf.feature_importances_

importance = pd.DataFrame([features, importance]).transpose()
importance = importance.rename(columns={0:'feature', 1:'importance'}).sort_values('importance', ascending=False)

In [30]:
importance

,feature,importance
2,nrhypos,0.278247
0,ngrams_last_mean,0.183312
4,depthfromtopsynset_normalised,0.167373
6,minwordlength,0.146328
5,glosslength_normalised,0.133383
8,polyscore_max,0.049269
7,nroflemmas,0.025633
3,nrpartrels_normalised,0.012891
1,nrdirhypers,0.003564
